<a href="https://colab.research.google.com/github/zelal-Eizaldeen/project2_Walmart_Price_Prediction/blob/main/project2_version2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import warnings
import statsmodels.api as sm
import patsy

from datetime import timedelta
import logging
import numpy as np
import pandas as pd
from time import time
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

References:

What We have tried 1: https://liangfgithub.github.io/Proj/F24_Proj2_hints_1_Python.html

What We have tried 2: https://liangfgithub.github.io/Proj/F24_Proj2_hints_2_Python.html

What we have learned 3: https://campuswire.com/c/GB46E5679/feed/457

performance target: 1580 or less; 1pt: above 1680

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
##Replace with your path to the Proj2_Data folder on GoogleColab
path_to_data='/content/drive/MyDrive/MastersDegree/CS598 PSL/Assignments/Projects/Project2/MyCode/Proj2_Data'
DATA_DIR ='/content/drive/MyDrive/MastersDegree/CS598 PSL/Assignments/Projects/Project2/MyCode/Proj2_Data'

In [4]:
# Configure logging
logging.basicConfig(
    filename=f'{DATA_DIR}/colab_log.log',
    level=logging.INFO,
    format='%(asctime)s - %(message)s',
    datefmt='%H:%M:%S'
)

# Suppress warnings
warnings.filterwarnings('ignore')

In [9]:
logging.info("Starting Walmart Price Prediction")

In [10]:
    """Train the model using SVD for smoothing."""
    start_time = time()
    num_folds = 10

In [11]:
i = 1

In [12]:
        fold_start = time()
        logging.info(f"Processing fold {i+1}/{num_folds}")
        test_pred = pd.DataFrame()

In [13]:
        # Read data for current fold
        train = pd.read_csv(f'{DATA_DIR}/fold_{i+1}/train.csv')
        test = pd.read_csv(f'{DATA_DIR}/fold_{i+1}/test.csv')

In [15]:
        # Convert dates to datetime
        train['Date'] = pd.to_datetime(train['Date'])
        test['Date'] = pd.to_datetime(test['Date'])

In [16]:
        # Extract week, year, and other useful features
        train['Wk'] = train['Date'].dt.isocalendar().week
        train['Yr'] = train['Date'].dt.year
        test['Wk'] = test['Date'].dt.isocalendar().week
        test['Yr'] = test['Date'].dt.year

        train = add_holiday_flags(train)
        test = add_holiday_flags(test)

In [24]:
train[train['Wk'] == 52]

,Store,Dept,Date,Weekly_Sales,IsHoliday,Wk,Yr,Is_SuperBowl,Is_Thanksgiving,Is_Christmas,Is_Pre_ChristmasEve,Is_After_Christmas
47,1,1,2010-12-31,19124.58,True,52,2010,0,0,1,0,1
112,1,2,2010-12-31,35819.83,True,52,2010,0,0,1,0,1
177,1,3,2010-12-31,8266.14,True,52,2010,0,0,1,0,1
242,1,4,2010-12-31,36913.80,True,52,2010,0,0,1,0,1
307,1,5,2010-12-31,32280.82,True,52,2010,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
190413,45,93,2010-12-31,2072.46,True,52,2010,0,0,1,0,1
190469,45,94,2010-12-31,3966.80,True,52,2010,0,0,1,0,1
190534,45,95,2010-12-31,43149.88,True,52,2010,0,0,1,0,1
190599,45,97,2010-12-31,5881.22,True,52,2010,0,0,1,0,1


In [25]:
        # Sort by Store, Dept, and Date for lag feature creation
        train = train.sort_values(['Store', 'Dept', 'Date'])

In [27]:
        # Create lagged features
        train['Lag_1'] = train.groupby(['Store', 'Dept'])['Weekly_Sales'].shift(1)
        train['Lag_2'] = train.groupby(['Store', 'Dept'])['Weekly_Sales'].shift(2)
        train['Rolling_Mean_4'] = train.groupby(['Store', 'Dept'])['Weekly_Sales'].transform(lambda x: x.rolling(4).mean())
        train.dropna(inplace=True)  # Drop rows with NaN lagged features

In [29]:
        departments = train['Dept'].unique()
        logging.info(f"Processing {len(departments)} departments for fold {i+1}")

In [30]:
        # Process each department
        for dept_idx, department in enumerate(departments, 1):
            if dept_idx % 5 == 0:  # Log every 5th department
                print(f"  Progress: {dept_idx}/{len(departments)} departments")

  Progress: 5/79 departments
  Progress: 10/79 departments
  Progress: 15/79 departments
  Progress: 20/79 departments
  Progress: 25/79 departments
  Progress: 30/79 departments
  Progress: 35/79 departments
  Progress: 40/79 departments
  Progress: 45/79 departments
  Progress: 50/79 departments
  Progress: 55/79 departments
  Progress: 60/79 departments
  Progress: 65/79 departments
  Progress: 70/79 departments
  Progress: 75/79 departments


In [31]:
department =1

In [37]:
            filtered_train = train[train['Dept'] == department]
            selected_columns = filtered_train[['Store',  'Date', 'Weekly_Sales']]


In [38]:
selected_columns

,Store,Date,Weekly_Sales
3,1,2010-02-26,19403.54
4,1,2010-03-05,21827.90
5,1,2010-03-12,21043.39
6,1,2010-03-19,22136.64
7,1,2010-03-26,26229.21
...,...,...,...
186379,45,2011-04-01,15734.83
186380,45,2011-04-08,21535.31
186381,45,2011-04-15,27964.64
186382,45,2011-04-22,64121.38


In [39]:
            # Create pivot table and perform SVD
            X_pivot = selected_columns.pivot(index='Store', columns='Date',
                                          values='Weekly_Sales').fillna(0)
            X_pivot

Date,2010-02-26,2010-03-05,2010-03-12,2010-03-19,2010-03-26,2010-04-02,2010-04-09,2010-04-16,2010-04-23,2010-04-30,...,2011-02-25,2011-03-04,2011-03-11,2011-03-18,2011-03-25,2011-04-01,2011-04-08,2011-04-15,2011-04-22,2011-04-29
Store,,,,,,,,,,,,,,,,,,,,,
1,19403.54,21827.90,21043.39,22136.64,26229.21,57258.43,42960.91,17596.96,16145.35,16555.11,...,19363.83,20327.61,21280.40,20334.23,20881.10,20398.09,23873.79,28762.37,50510.31,41512.39
2,25962.32,27372.05,28660.87,28446.92,32213.99,82766.07,66560.00,21308.59,22163.12,24066.20,...,22631.87,25562.33,26025.39,25405.55,23880.58,24591.68,28223.45,34644.85,72065.08,57905.55
3,4992.00,5172.73,5540.28,5118.12,6620.12,16399.01,8316.05,4061.89,4417.17,4159.25,...,4896.15,6673.83,6440.67,5850.92,6692.63,6389.96,6885.58,9203.25,15862.38,9410.55
4,30107.54,31580.69,29452.49,30853.27,33401.19,75535.18,56416.66,27744.54,23961.80,25076.00,...,26289.28,26562.46,31284.95,32981.01,32193.44,33809.56,36416.80,44011.29,83445.89,50759.95
5,7168.41,8344.13,7531.45,8719.27,9945.67,21113.02,15310.29,6815.91,7178.81,7582.13,...,9676.42,8743.51,8702.11,8615.19,8809.64,9287.00,9381.66,12239.56,18238.83,12830.22
6,19896.08,22839.36,24071.96,26866.90,28291.37,62544.57,44695.23,20110.25,19254.91,18597.43,...,16088.17,16806.47,18631.32,20208.62,18217.57,21313.31,23306.19,26876.39,54711.64,41404.80
7,8602.73,9541.12,9538.70,11725.85,11452.69,20518.44,13658.16,6891.66,5903.69,6584.55,...,8532.36,8563.14,8719.24,9231.70,8861.84,8263.41,8459.93,8560.97,15486.48,12575.17
8,11722.71,12979.74,12305.83,12520.30,15764.25,36979.10,25305.29,10141.01,9827.30,10534.42,...,12309.14,13304.63,13454.88,13328.59,13420.29,13644.98,15975.88,18562.34,33591.13,22609.44
9,10530.98,10438.47,11004.74,9813.87,11402.28,24205.64,13421.94,8926.10,8462.08,8723.98,...,8805.35,9177.05,9588.53,9387.59,9909.43,10393.72,11515.04,13693.33,24940.66,17649.37


In [40]:
            # this is actually Y centered
            X_matrix = X_pivot.values
            store_mean = X_matrix.mean(axis=1, keepdims=True)
            X_centered = X_matrix - store_mean
            X_centered

array([[-4089.66483871, -1665.30483871, -2449.81483871, ...,
         5269.16516129, 27017.10516129, 18019.18516129],
       [-6176.28516129, -4766.55516129, -3477.73516129, ...,
         2506.24483871, 39926.47483871, 25766.94483871],
       [-1638.19241935, -1457.46241935, -1089.91241935, ...,
         2573.05758065,  9232.18758065,  2780.35758065],
       ...,
       [-1142.32903226,  -663.80903226,  -321.24903226, ...,
          845.17096774,  3197.91096774,  1249.18096774],
       [-1527.82516129, -1421.93516129, -1310.95516129, ...,
         3271.90483871,  5509.01483871,  2118.60483871],
       [-4462.66016129, -2509.87016129, -1497.35016129, ...,
         8845.89983871, 45002.63983871, 19451.55983871]])

In [41]:
            # SVD computation
            U, D, Vt = np.linalg.svd(X_centered, full_matrices=False)

In [42]:
D

array([635902.1455027 ,  96828.84336782,  77526.83962678,  59075.79428377,
        38561.63149644,  34127.9271799 ,  30771.40126418,  24537.8325245 ,
        22832.82855343,  20015.39441071,  17881.08372837,  17131.16242178,
        16117.92915673,  13975.11816924,  12907.08093431,  12519.56933338,
        11165.51323949,  10504.40050258,   8828.15431786,   8727.80356016,
         8062.85177376,   7271.44103029,   6371.02834402,   6203.03652129,
         6077.78360351,   5688.59191941,   4975.05099892,   4728.25177848,
         4631.29535131,   4313.68219149,   3760.70812613,   3530.28878454,
         3302.93525242,   2985.38579738,   2856.69777565,   2676.97117877,
         2498.2434925 ,   2279.9992044 ,   2078.59967306,   1830.6548106 ,
         1645.15371049,   1319.88759448,   1218.22370252,    951.30706139,
          752.90707387])

In [43]:
            n_comp = 8
            D_tilda = np.zeros_like(D)
            D_tilda

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [44]:
            D_tilda[:n_comp] = D[:n_comp]
            D_tilda

array([635902.1455027 ,  96828.84336782,  77526.83962678,  59075.79428377,
        38561.63149644,  34127.9271799 ,  30771.40126418,  24537.8325245 ,
            0.        ,      0.        ,      0.        ,      0.        ,
            0.        ,      0.        ,      0.        ,      0.        ,
            0.        ,      0.        ,      0.        ,      0.        ,
            0.        ,      0.        ,      0.        ,      0.        ,
            0.        ,      0.        ,      0.        ,      0.        ,
            0.        ,      0.        ,      0.        ,      0.        ,
            0.        ,      0.        ,      0.        ,      0.        ,
            0.        ,      0.        ,      0.        ,      0.        ,
            0.        ,      0.        ,      0.        ,      0.        ,
            0.        ])

In [45]:
            X_tilda = U[:, :n_comp] @ np.diag(D_tilda[:n_comp]) @ Vt[:n_comp, :]

In [46]:
            X_smoothed = X_tilda + store_mean

In [47]:
            # Convert back to DataFrame
            X_smoothed_df = pd.DataFrame(X_smoothed, index=X_pivot.index,
                                       columns=X_pivot.columns).reset_index()

            X_original_format = X_smoothed_df.melt(id_vars=['Store'],
                                                 var_name='Date',
                                                 value_name='Weekly_Sales')

In [48]:
X_original_format

,Store,Date,Weekly_Sales
0,1,2010-02-26 00:00:00,19587.122007
1,2,2010-02-26 00:00:00,25839.241900
2,3,2010-02-26 00:00:00,5394.322360
3,4,2010-02-26 00:00:00,27625.520005
4,5,2010-02-26 00:00:00,8472.423889
...,...,...,...
2785,41,2011-04-29 00:00:00,30328.162581
2786,42,2011-04-29 00:00:00,15347.457488
2787,43,2011-04-29 00:00:00,7884.735744
2788,44,2011-04-29 00:00:00,7973.287073


In [49]:
null_values = X_original_format.isnull().sum()

In [50]:
            X_original_format['Dept'] = department
            X_original_format = X_original_format.sort_values(
                by=['Store', 'Date']).reset_index(drop=True)


In [51]:
X_original_format

,Store,Date,Weekly_Sales,Dept
0,1,2010-02-26 00:00:00,19587.122007,1
1,1,2010-03-05 00:00:00,21132.339446,1
2,1,2010-03-12 00:00:00,20553.874803,1
3,1,2010-03-19 00:00:00,21047.613069,1
4,1,2010-03-26 00:00:00,23756.959437,1
...,...,...,...,...
2785,45,2011-04-01 00:00:00,16918.989380,1
2786,45,2011-04-08 00:00:00,21388.498547,1
2787,45,2011-04-15 00:00:00,27580.501707,1
2788,45,2011-04-22 00:00:00,61870.054080,1


In [53]:
            # Prepare train-test pairs
            train_pairs = X_original_format[['Store', 'Dept']].drop_duplicates(
                ignore_index=True)
            test_pairs = test[['Store', 'Dept']].drop_duplicates(ignore_index=True)
            unique_pairs = pd.merge(train_pairs, test_pairs,
                                  how='inner', on=['Store', 'Dept'])

In [54]:
            # Process training data
            train_split = unique_pairs.merge(train, on=['Store', 'Dept'], how='left')
            train_split = preprocess(train_split)

In [55]:
train_split

,Store,Dept,Date,Weekly_Sales,IsHoliday,Wk,Yr,Is_SuperBowl,Is_Thanksgiving,Is_Christmas,Is_Pre_ChristmasEve,Is_After_Christmas,Lag_1,Lag_2,Rolling_Mean_4,Yr2
0,1,1,2010-02-26,19403.54,0,8,2010,0,0,0,0,0,41595.55,46039.49,32990.7700,4040100
1,1,1,2010-03-05,21827.90,0,9,2010,0,0,0,0,0,19403.54,41595.55,32216.6200,4040100
2,1,1,2010-03-12,21043.39,0,10,2010,0,0,0,0,0,21827.90,19403.54,25967.5950,4040100
3,1,1,2010-03-19,22136.64,0,11,2010,0,0,0,0,0,21043.39,21827.90,21102.8675,4040100
4,1,1,2010-03-26,26229.21,0,12,2010,0,0,0,0,0,22136.64,21043.39,22809.2850,4040100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2785,45,1,2011-04-01,15734.83,0,13,2011,0,0,0,0,0,16190.45,14587.49,14701.9400,4044121
2786,45,1,2011-04-08,21535.31,0,14,2011,0,0,0,0,0,15734.83,16190.45,17012.0200,4044121
2787,45,1,2011-04-15,27964.64,0,15,2011,0,0,0,0,0,21535.31,15734.83,20356.3075,4044121
2788,45,1,2011-04-22,64121.38,0,16,2011,0,0,0,0,0,27964.64,21535.31,32339.0400,4044121


In [56]:
            # Create model matrices
            X = patsy.dmatrix('Weekly_Sales + Store + Dept  + IsHoliday+ Yr + Yr2 + Wk',
                              data = train_split,
                              return_type='dataframe')

In [57]:
X

,Intercept,Wk[T.2],Wk[T.3],Wk[T.4],Wk[T.5],Wk[T.6],Wk[T.7],Wk[T.8],Wk[T.9],Wk[T.10],...,Wk[T.49],Wk[T.50],Wk[T.51],Wk[T.52],Weekly_Sales,Store,Dept,IsHoliday,Yr,Yr2
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,19403.54,1.0,1.0,0.0,2010.0,4040100.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,21827.90,1.0,1.0,0.0,2010.0,4040100.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,21043.39,1.0,1.0,0.0,2010.0,4040100.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,22136.64,1.0,1.0,0.0,2010.0,4040100.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,26229.21,1.0,1.0,0.0,2010.0,4040100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2785,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,15734.83,45.0,1.0,0.0,2011.0,4044121.0
2786,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,21535.31,45.0,1.0,0.0,2011.0,4044121.0
2787,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,27964.64,45.0,1.0,0.0,2011.0,4044121.0
2788,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,64121.38,45.0,1.0,0.0,2011.0,4044121.0


In [58]:
            train_split = dict(tuple(X.groupby(['Store', 'Dept'])))

In [59]:
train_split

{(1.0,
  1.0):     Intercept  Wk[T.2]  Wk[T.3]  Wk[T.4]  Wk[T.5]  Wk[T.6]  Wk[T.7]  Wk[T.8]  \
 0         1.0      0.0      0.0      0.0      0.0      0.0      0.0      1.0   
 1         1.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
 2         1.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
 3         1.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
 4         1.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
 ..        ...      ...      ...      ...      ...      ...      ...      ...   
 57        1.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
 58        1.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
 59        1.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
 60        1.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
 61        1.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
 
     Wk[T.9]

In [5]:
# Feature Engineering
def preprocess(data):
    """Preprocess the data by handling missing values and creating time-based features."""
    data.fillna(0, inplace=True)
    tmp = pd.to_datetime(data['Date'])
    data['Wk'] = tmp.dt.isocalendar().week
    data['Yr'] = tmp.dt.year
    data['Yr2'] = data.Yr ** 2
    data['Wk'] = pd.Categorical(data['Wk'], categories=[i for i in range(1, 53)])
    data['IsHoliday'] = data['IsHoliday'].apply(int)
    return data

In [6]:
# Function to mark holiday weeks
def add_holiday_flags(df):
    df['Is_SuperBowl'] = (df['Wk'] == 6).astype(int)
    df['Is_Thanksgiving'] = (df['Wk'] == 47).astype(int)
    df['Is_Christmas'] = (df['Wk'] == 52).astype(int)

    df['Is_Pre_ChristmasEve'] = (
         ((df['Date'] >= pd.to_datetime('2010-12-23')) & (df['Date'] - pd.Timedelta(days=7) < pd.to_datetime('2010-12-23')))
        |((df['Date'] >= pd.to_datetime('2011-12-23')) & (df['Date'] - pd.Timedelta(days=7) < pd.to_datetime('2011-12-23')))
        |((df['Date'] >= pd.to_datetime('2012-12-23')) & (df['Date'] - pd.Timedelta(days=7) < pd.to_datetime('2012-12-23')))
                                ).astype(int)
    df['Is_After_Christmas'] = (
         ((df['Date'] >= pd.to_datetime('2010-12-26')) & (df['Date'] - pd.Timedelta(days=7) < pd.to_datetime('2010-12-26')))
        |((df['Date'] >= pd.to_datetime('2011-12-26')) & (df['Date'] - pd.Timedelta(days=7) < pd.to_datetime('2011-12-26')))
        |((df['Date'] >= pd.to_datetime('2012-12-26')) & (df['Date'] - pd.Timedelta(days=7) < pd.to_datetime('2012-12-26')))
                                ).astype(int)
    return df

In [7]:
def train_svd():
    """Train the model using SVD for smoothing."""
    start_time = time()
    num_folds = 10

    for i in range(num_folds):
        fold_start = time()
        logging.info(f"Processing fold {i+1}/{num_folds}")
        test_pred = pd.DataFrame()

        # Read data for current fold
        train = pd.read_csv(f'{DATA_DIR}/fold_{i+1}/train.csv')
        test = pd.read_csv(f'{DATA_DIR}/fold_{i+1}/test.csv')

                # Convert dates to datetime
        train['Date'] = pd.to_datetime(train['Date'])
        test['Date'] = pd.to_datetime(test['Date'])

        # Extract week, year, and other useful features
        train['Wk'] = train['Date'].dt.isocalendar().week
        train['Yr'] = train['Date'].dt.year
        test['Wk'] = test['Date'].dt.isocalendar().week
        test['Yr'] = test['Date'].dt.year

        train = add_holiday_flags(train)
        test = add_holiday_flags(test)

        # Sort by Store, Dept, and Date for lag feature creation
        train = train.sort_values(['Store', 'Dept', 'Date'])

        # Create lagged features
        train['Lag_1'] = train.groupby(['Store', 'Dept'])['Weekly_Sales'].shift(1)
        train['Lag_2'] = train.groupby(['Store', 'Dept'])['Weekly_Sales'].shift(2)
        train['Rolling_Mean_4'] = train.groupby(['Store', 'Dept'])['Weekly_Sales'].transform(lambda x: x.rolling(4).mean())
        train.dropna(inplace=True)  # Drop rows with NaN lagged features


        departments = train['Dept'].unique()
        logging.info(f"Processing {len(departments)} departments for fold {i+1}")

        # Process each department
        for dept_idx, department in enumerate(departments, 1):
            if dept_idx % 5 == 0:  # Log every 5th department
                logging.info(f"  Progress: {dept_idx}/{len(departments)} departments")

            filtered_train = train[train['Dept'] == department]
            selected_columns = filtered_train[['Store', 'Date', 'Weekly_Sales']]

            # Create pivot table and perform SVD
            X_pivot = selected_columns.pivot(index='Store', columns='Date',
                                          values='Weekly_Sales').fillna(0)
            X_matrix = X_pivot.values
            store_mean = X_matrix.mean(axis=1, keepdims=True)
            X_centered = X_matrix - store_mean

            # SVD computation
            U, D, Vt = np.linalg.svd(X_centered, full_matrices=False)
            n_comp = 8
            D_tilda = np.zeros_like(D)
            D_tilda[:n_comp] = D[:n_comp]
            X_tilda = U[:, :n_comp] @ np.diag(D_tilda[:n_comp]) @ Vt[:n_comp, :]
            X_smoothed = X_tilda + store_mean


            # Convert back to DataFrame
            X_smoothed_df = pd.DataFrame(X_smoothed, index=X_pivot.index,
                                       columns=X_pivot.columns).reset_index()

            X_original_format = X_smoothed_df.melt(id_vars=['Store'],
                                                 var_name='Date',
                                                 value_name='Weekly_Sales')

            null_values = X_original_format.isnull().sum()

            X_original_format['Dept'] = department
            X_original_format = X_original_format.sort_values(
                by=['Store', 'Date']).reset_index(drop=True)



            # Prepare train-test pairs
            train_pairs = X_original_format[['Store', 'Dept']].drop_duplicates(
                ignore_index=True)
            test_pairs = test[['Store', 'Dept']].drop_duplicates(ignore_index=True)
            unique_pairs = pd.merge(train_pairs, test_pairs,
                                  how='inner', on=['Store', 'Dept'])

            # Process training data
            train_split = unique_pairs.merge(train, on=['Store', 'Dept'], how='left')
            train_split = preprocess(train_split)

            # Create model matrices
            X = patsy.dmatrix('Weekly_Sales + Store + Dept  + IsHoliday+ Yr + Yr2 + Wk + Is_Pre_ChristmasEve + Is_After_Christmas',
                              data = train_split,
                              return_type='dataframe')



            train_split = dict(tuple(X.groupby(['Store', 'Dept'])))

            # Process test data
            test_split = unique_pairs.merge(test, on=['Store', 'Dept'], how='left')
            test_split = preprocess(test_split)



            X = patsy.dmatrix('Store + Dept + IsHoliday +Yr + Yr2 + Wk + Is_Pre_ChristmasEve + Is_After_Christmas',
                                data = test_split,
                                return_type='dataframe')

            X['Date'] = test_split['Date']
            test_split = dict(tuple(X.groupby(['Store', 'Dept'])))
            keys = list(train_split)



            # Train and predict for each store-department combination
            for key in keys:
                X_train = train_split[key]
                X_test = test_split[key]

                Y = X_train['Weekly_Sales']
                X_train = X_train.drop(['Weekly_Sales','Store', 'Dept','IsHoliday'], axis=1)

                cols_to_drop = X_train.columns[(X_train == 0).all()]
                X_train = X_train.drop(columns=cols_to_drop)
                X_test = X_test.drop(columns=cols_to_drop)
                cols_to_drop = []
                for j in range(len(X_train.columns) - 1, 1, -1):  # Start from the last column and move backward
                    col_name = X_train.columns[j]
                    # Extract the current column and all previous columns
                    tmp_Y = X_train.iloc[:, j].values
                    tmp_X = X_train.iloc[:, :j].values

                    coefficients, residuals, rank, s = np.linalg.lstsq(tmp_X, tmp_Y, rcond=None)
                    if np.sum(residuals) < 1e-16:
                            cols_to_drop.append(col_name)
                X_train = X_train.drop(columns=cols_to_drop)
                X_test = X_test.drop(columns=cols_to_drop)

                model = sm.OLS(Y, X_train).fit()
                mycoef = model.params.fillna(0)

                # Make predictions
                tmp_pred = X_test[['Store', 'Dept', 'Date', 'IsHoliday']]
                X_test = X_test.drop(['Store', 'Dept', 'Date', 'IsHoliday'], axis=1, errors='ignore')

                tmp_pred['Weekly_Pred'] =  np.dot(X_test, mycoef)
                test_pred = pd.concat([test_pred, tmp_pred], ignore_index=True)

            test_pred['Weekly_Pred'].fillna(0, inplace=True)
            test_pred.to_csv(f'{DATA_DIR}/fold_{i+1}/mypred.csv', index=False)


                # Fit Gradient Boosting model
                # model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=0)
                # model.fit(X_train, Y)

        fold_time = time() - fold_start
        logging.info(f"Completed fold {i+1} in {fold_time:.1f} seconds")

    total_time = time() - start_time
    logging.info(f"Completed all folds in {total_time:.1f} seconds")

In [8]:
def evaluate():
    """Evaluate the model's predictions."""
    logging.info("Starting evaluation...")
    test_with_label = pd.read_csv(f'{DATA_DIR}/test_with_label.csv')
    num_folds = 10
    wae = []

    for i in range(num_folds):
        # Read test data and predictions
        test = pd.read_csv(f'{DATA_DIR}/fold_{i+1}/test.csv')
        test = test.merge(test_with_label, on=['Date', 'Store', 'Dept'])
        test_pred = pd.read_csv(f'{DATA_DIR}/fold_{i+1}/mypred.csv')

        # Merge and calculate weighted absolute error
        new_test = test.merge(test_pred, on=['Date', 'Store', 'Dept'], how='left')
        actuals = new_test['Weekly_Sales'].fillna(0)
        preds = new_test['Weekly_Pred'].fillna(0)
        weights = new_test['IsHoliday_x'].apply(lambda x: 5 if x else 1)

        wae_score = sum(weights * abs(actuals - preds)) / sum(weights)
        wae.append(wae_score)
        logging.info(f"Fold {i+1} WAE: {wae_score:.3f}")

    avg_wae = sum(wae) / len(wae)
    logging.info(f"Average WAE across all folds: {avg_wae:.3f}")

    return wae

In [ ]:
logging.info("Model: adjustment: add two new variables: Is_Pre_ChristmasEve + Is_After_Christmas ")
def main():
    """Main function to run the Walmart price prediction model."""
    start_time = time()
    logging.info("Starting Walmart Price Prediction")

    logging.info("Training models with SVD smoothing...")
    train_svd()

    logging.info("Evaluating predictions...")
    wae = evaluate()

    total_time = time() - start_time
    logging.info(f"Processing complete! Total time: {total_time:.1f} seconds")

if __name__ == "__main__":
    main()


In [ ]:
with open('logfile.log', 'r') as file:
    log_contents = file.read()
    print(log_contents)  # This will print out the contents of 'logfile.log'